In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
# load data
dataset = pd.read_csv('../input/train_data.csv')
dataset.head 

In [ ]:
import featuretools as ft
import matplotlib.pyplot as plt
volume_head = ['BidVolume1','AskVolume1']

i = 0
print(i)
X = []
Y = []
while (i < len(dataset)-30):
    if (dataset['Date'].iloc[i]!=dataset['Date'].iloc[i+30]):
        i = i+30
    else:
        x_item = np.array(dataset[volume_head].iloc[i:i+10])
        x_item = x_item.reshape(20).tolist()
        X.append(x_item)
#         print(x_item)
        y_item = sum(dataset['MidPrice'].iloc[i+10:i+30]) / 20 - dataset['MidPrice'].iloc[i+9]
        Y.append(y_item)
        i = i+10
print(len(X))
print(len(Y))

In [ ]:
print(X[0])
print(Y[0])

In [ ]:
from sklearn.model_selection import train_test_split
X_array = np.array(X)
X_array[:,:]=np.log10(X_array[:,:])

Y_array = np.array(Y)
Y_array[:] = Y_array[:] * 100

print(X_array)
print(Y_array)
seed = 6
test_size = 0.0001
X_train, X_test, y_train, y_test = train_test_split(X_array, Y_array, test_size=test_size, random_state=seed)

plt.hist(y_train)
plt.hist(y_test)

In [ ]:
from keras.layers import Activation, Dropout, Dense
from keras.models import Sequential

model = Sequential()
model.add(Dense(512,input_shape = (len(X_train[0]),), activation='tanh'))
# model.add(Dropout(0.125))
model.add(Dense(128, activation = 'sigmoid'))
# model.add(Dropout(0.125))
model.add(Dense(16, activation = 'relu'))
model.add(Dense(1, activation = 'tanh'))
model.compile(loss='mse',optimizer = 'adam')
model.summary()

In [ ]:
model.fit(X_train, y_train, epochs = 30,batch_size = 32)

In [ ]:
import math
eval_loss = model.evaluate(x=X_test, y=y_test, batch_size=None, verbose=1, sample_weight=None, steps=None)
print(math.sqrt(eval_loss))
print(eval_loss, y_test)

In [ ]:
test_data = pd.read_csv("../input/test_data.csv")

ans_hist = []
ans_list = []
ans_csv = pd.DataFrame(columns=['caseid','midprice'])
for i in range(142,1000):
    standard_price = test_data['MidPrice'].iloc[i*10+9]

    
    x_item = np.array(test_data[volume_head].iloc[i*10:i*10+10])
    x_item[:] = np.log10(x_item[:])
    x_item = x_item.reshape(1,20)
#     print(x_item)
#     y_item = sum(dataset['MidPrice'].iloc[i+10:i+30]) / 20 - dataset['MidPrice'].iloc[i+9]
    y_item = model.predict(x_item)
    y_ans = y_item[0,0]
    ans_hist.append(y_ans)
    ans_list.append(y_ans/100 + standard_price)
# print(ans_hist)
plt.hist(ans_hist)
ans_csv.to_csv('data.csv', index = False, index_label = False)
print(ans_list)